In [ ]:
#@title Configuration Parameters

# NOTE: hyper-parameters must be tunned for best performance

#@markdown General Parameters
verbose = True        #@param {type: "boolean"}
seed = 1234           #@param {type: "integer"}

#@markdown DeepONet parameters
m = 200                  #@param {type: "integer"} # of sensors  
q = 10                   #@param {type: "integer"} # of sampling y's in Y
n_basis = 100            #@param {type: "integer"} # of basis functions  
branch_type = "modified" #@param ["modified", "MLP"]
trunk_type = "modified"  #@param ["modified", "MLP"]
width = 200              #@param {type: "integer"}
depth = 3                #@param {type: "integer"}
activation = "sin"       #@param ["leaky", "silu", "Rrelu", "Mish", "sin", "relu", "tanh", "selu", "gelu"]

#@markdown training parameters
learning_rate = 5e-5     #@param {type: "raw"}
batch_size = 1000        #@param {type: "integer"}
n_epochs = 5000          #@param {type: "integer"}

#@markdown Data parameters
version = "v1"
state = "voltage"
cont = "mix"

In [ ]:
#@title Imports
import numpy as np
import torch

import utils.pytorch_utils as ptu

from models.nns import prob_DeepONet
from training.supervisor import probabilistic_train
from utils.data_structures import data_generator, get_data, get_traj_data
from utils.utils import compute_metrics, fraction_in_CI, l1_relative_error, l2_relative_error, plot_pred_UQ, test, test_one, trajectory_rel_error, update_metrics_history

In [ ]:
#@title Main
###################################
# Step 1: initialize the device
###################################
ptu.init_gpu(verbose=verbose)

###################################
# Step 2: initialize the device
###################################
np.random.seed(seed=seed)
torch.manual_seed(seed)

Using GPU id 0


In [ ]:
###################################
# Step 3: get training and test trajectory data
###################################
train_path = "./data/trustworthydataset" + version + "/train-data-" + state + "-m-" + str(m) + "-Q-" + str(q) + "-" + cont + ".npz"
test_path = "./data/trustworthydataset" + version + "/test-data-" + state + "-m-" + str(m) + "-" + cont + ".npz"

u_train, y_train, s_train, t_sim = get_data(train_path, verbose=verbose)
u_test, y_test, s_test = get_traj_data(test_path, verbose=verbose)

if verbose:
    print("the number of testing trajectories is {}".format(len(u_test)))

Shapes are: u (17490, 200), y (17490, 1), and s (17490, 1)
Shapes are: u (600, 200), y (600, 1), and s (600, 1)
the number of testing trajectories is 750


In [ ]:
###################################
# Step 4: move data to torch
###################################
u_torch_train = torch.from_numpy(u_train).float().to(ptu.device)
y_torch_train = torch.from_numpy(y_train).float().to(ptu.device)
s_torch_train = torch.from_numpy(s_train).float().to(ptu.device)

# test data
u_torch_test = []
y_torch_test = []

for k in range(len(u_test)):
    u_torch_test.append(torch.from_numpy(u_test[k]).float().to(ptu.device))
    y_torch_test.append(torch.from_numpy(y_test[k]).float().to(ptu.device))

###################################
# Step 5: define torch dataset
###################################
torch_data = data_generator(u_torch_train, y_torch_train, s_torch_train)

In [ ]:
###################################
# Step 6: build prob-DeepONet
###################################
dim = 1
n_sensors = m

branch = {}
branch["type"] = branch_type
branch["layer_size"] = [n_sensors] + [width] * depth + [n_basis]
branch["activation"] = activation

trunk = {}
trunk["type"] = trunk_type
trunk["layer_size"] = [dim] + [width] * depth + [n_basis]
trunk["activation"] = activation

model = prob_DeepONet(branch, trunk).to(ptu.device)

if verbose:
    print(model)

prob_DeepONet(
  (branch): modified_MLP(
    (net): ModuleList(
      (0): Linear(in_features=200, out_features=200, bias=True)
      (1): Linear(in_features=200, out_features=200, bias=True)
    )
    (U): Sequential(
      (0): Linear(in_features=200, out_features=200, bias=True)
    )
    (V): Sequential(
      (0): Linear(in_features=200, out_features=200, bias=True)
    )
    (activation): sin_act()
  )
  (trunk): modified_MLP(
    (net): ModuleList(
      (0): Linear(in_features=1, out_features=200, bias=True)
      (1): Linear(in_features=200, out_features=200, bias=True)
    )
    (U): Sequential(
      (0): Linear(in_features=1, out_features=200, bias=True)
    )
    (V): Sequential(
      (0): Linear(in_features=1, out_features=200, bias=True)
    )
    (activation): sin_act()
  )
  (branch_mu): Sequential(
    (0): sin_act()
    (1): Linear(in_features=200, out_features=200, bias=True)
    (2): sin_act()
    (3): Linear(in_features=200, out_features=100, bias=True)
  )
  (br

In [ ]:
###################################
# Step 7: define training parameters
###################################
train_params = {}
train_params["learning rate"] = learning_rate
train_params["batch size"] = batch_size
train_params["epochs"] = n_epochs
train_params["print every"] = 10
train_params["eval every"] = 1

###################################
# Step 8: define scheduler parameters
###################################
scheduler_params = {}
scheduler_params["patience"] = 1000
scheduler_params["factor"] = 0.8

###################################
# Step 9: define metrics and losses
###################################
metrics = [l1_relative_error, l2_relative_error]
L1_history = {}
L1_history["max"] = []
L1_history["min"] = []
L1_history["mean"] = []
L2_history = {}
L2_history["max"] = []
L2_history["min"] = []
L2_history["mean"] = []

###################################
# Step 10: initial test
###################################
n_test_samples = 100
u_torch_test_100 = u_torch_test[:n_test_samples]
y_torch_test_100 = y_torch_test[:n_test_samples]
s_test_100 = s_test[:n_test_samples]

# we perform an initial test for the probabilistic model
s_pred, sigma_pred = test(model, u_torch_test_100, y_torch_test_100)
metrics_state = compute_metrics(s_test_100, s_pred, metrics, verbose=verbose)
L1_history = update_metrics_history(L1_history, metrics_state[0])
L2_history = update_metrics_history(L2_history, metrics_state[1])
del metrics_state

l1-relative errors: max=407.816, min=268.787, mean=367.748
l2-relative errors: max=422.025, min=283.055, mean=383.488


In [ ]:
###################################
# Step 11: training
###################################

logging_file = "./output/best-model.pt"

logger, loss_history = probabilistic_train(
    model,
    torch_data,
    train_params,
    scheduler_params=scheduler_params,
    verbose=verbose,
    test_data=(u_torch_test_100, y_torch_test_100, s_test_100),
    loss_history=(L1_history, L2_history),
    metrics=metrics,
    logging_file=logging_file,
    )


***** Probabilistic Training for 5000 epochs and using 17490 data samples*****



  0%|          | 0/5000 [00:00<?, ?it/s]

In [ ]:
#@title Postprocessing
import matplotlib.pyplot as plt 

plt.figure()
plt.plot(logger["prob loss"])
plt.xlabel("epoch")
plt.ylabel("loss")
plt.show()

# L1 plot
plt.figure()
plt.plot(L1_history["mean"], "r-", label = "Probabilistic")
plt.legend(framealpha=1, frameon=False)
plt.xlabel("epochs")
plt.ylabel("$L^1$ mean relative error %")
plt.title("$L^1$ error")
plt.show()

# L2 plot
plt.figure()
plt.plot(L2_history["mean"], "r-", label = "Probabilistic")
plt.legend(framealpha=1, frameon=False)
plt.xlabel("epochs")
plt.ylabel("$L^2$ mean relative error %")
plt.title("$L^2$ error")
plt.show()

In [ ]:
#@title Testing and UQ
###################################
# Step 1: restore best model
###################################
check_point = ptu.restore(logging_file)
state_dict = check_point['state_dict']
model.load_state_dict(state_dict)
model.to(ptu.device);

L1_test = []
L2_test = []
mean_test = []
std_test = []
ratios = []

###################################
# Step 2: define input and number of test trajectories
###################################
t_input = np.linspace(0, 2.0, num=m)[:, None]
n_test_trajs = 100
plot_num_traj = [1, 35, 48]

In [ ]:
###################################
# Step 3: compute errors and plot confidence intervals
###################################
for k in range(n_test_trajs):
    mean_k, std_k = test_one(model, u_torch_test[k], y_torch_test[k]) 
    mean_test.append(mean_k)
    std_test.append(std_k)
    
    if k in plot_num_traj:
        plot_pred_UQ(t_input, u_test[k][0,:], y_test[k], s_test[k], mean_k, std_k, xlabel="$time~(s)$", ylabel="voltage (p.u.)", v_lims=False)
        l1_error, l2_error = trajectory_rel_error(s_test[k], mean_k, verbose=True)
        ratio = fraction_in_CI(s_test[k], mean_k, std_k, verbose=True)        
    else:
        l1_error, l2_error = trajectory_rel_error(s_test[k], mean_k, verbose=False)
        ratio = fraction_in_CI(s_test[k], mean_k, std_k, verbose=False)
    L1_test.append(l1_error)
    L2_test.append(l2_error)
    ratios.append(ratio)
    
if verbose:
    mean_L1 = np.round(100 * np.mean(L1_test), decimals=5)
    mean_L2 = np.round(100 * np.mean(L2_test), decimals=5)
    std_L1 = np.round(100 * np.std(L1_test), decimals=5)
    std_L2 = np.round(100 * np.std(L2_test), decimals=5)
    print("Best mean relative errors: L1={} and L2={}".format(mean_L1, mean_L2))
    print("Standard deviation of relative errors: L1={} and L2={}".format(std_L1, std_L2))
    
if verbose:
    print("ratios-[max, min, mean] = [{},{},{}]".format(100 * np.max(ratios), 100 * np.min(ratios), 100 * np.mean(ratios)))